In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [4]:
import numpy as np
np.bool = np.bool_

In [5]:
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 

In [6]:
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [7]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [8]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .enableHiveSupport()
    .getOrCreate()
)

25/02/17 18:59:43 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
25/02/17 18:59:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ec13aabc-5d55-483d-9312-d330417380dc;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-avro_2.12;3.5.0 in central
	found org.tukaani#xz;1.9 in central
	found io.delta#delta-iceberg_2.12;3.0.0 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
:: resolution report :: resolve 248ms :: artifacts dl 15ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.3 from central in [default]
	com.google.errorprone#error_prone_annotations;2.10.0 from central in [default]
	io.delta#delta-iceberg_2.12;3.0.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 

In [13]:
base_path = f"{hdfs_lakehouse_base_path}/bronze/trump_btc"

# List of folder names (from the structure provided)
folder_names = ["BTC", "Trump"]

# Dictionary to store DataFrames
dataframes = {}

# Loop through each folder and read the CSV file inside it
for folder_name in folder_names:
    # Construct the file path
    file_path = os.path.join(base_path, folder_name)
    
    # Read the CSV file into a DataFrame
    dataframes[folder_name] = (spark.read
                                        .option("header", "true")
                                        .option("inferSchema", "true")
                                        .csv(file_path))
    print(f"Loaded DataFrame for {folder_name}_raw")

# Example: Accessing one of the DataFrames
btc_df = dataframes["BTC"]
trump_df=dataframes["Trump"]


Loaded DataFrame for BTC_raw
Loaded DataFrame for Trump_raw


In [14]:
btc_df.toPandas()

,Date,Close,High,Low,Open,Volume
0,None,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
1,2014-09-17,457.3340148925781,468.17401123046875,452.4219970703125,465.864013671875,21056800
2,2014-09-18,424.44000244140625,456.8599853515625,413.10400390625,456.8599853515625,34483200
3,2014-09-19,394.7959899902344,427.8349914550781,384.5320129394531,424.1029968261719,37919700
4,2014-09-20,408.90399169921875,423.2959899902344,389.88299560546875,394.6730041503906,36863600
...,...,...,...,...,...,...
3753,2024-12-25,99299.1953125,99478.75,97593.46875,98675.9140625,33700394629
3754,2024-12-26,95795.515625,99884.5703125,95137.8828125,99297.6953125,47054980873
3755,2024-12-27,94164.859375,97294.84375,93310.7421875,95704.9765625,52419934565
3756,2024-12-28,95163.9296875,95525.8984375,94014.2890625,94160.1875,24107436185


In [15]:
trump_df.toPandas()

,date,favorites,id,isRetweet,retweets,text
0,1730391364149,8601.0,1.134029e+17,False,2282.0,"""<p>I would like to wish our great Bitcoiners a Happy 16th Anniversary of Satoshi’s White Paper. We will end Kamala’s war on crypto, &amp; Bitcoin will be MADE IN THE USA! VOTE TRUMP! <a href=""""https://truthsocial.com/tags/Bitcoin"""" class=""""mention hashtag"""" rel=""""tag"""">#<span>Bitcoin</span></a> <a href=""""https://truthsocial.com/tags/FreeRossDayOne"""" class=""""mention hashtag"""" rel=""""tag"""">#<span>FreeRossDayOne</span></a></p>"""
1,1727709073460,8655.0,1.132271e+17,False,2516.0,"""<p>I promised to Make America Great Again, this time with crypto. <a href=""""https://truthsocial.com/tags/WorldLibertyFi"""" class=""""mention hashtag"""" rel=""""tag"""">#<span>WorldLibertyFi</span></a> is planning to help make America the crypto capital of the world! The whitelist for eligible persons is officially open – this is your chance to be part of this historic moment. Join: <a href=""""https://links.truthsocial.com/link/113073993409242797"""" rel=""""nofollow noopener noreferrer"""" target=""""_blank""""><span class=""""invisible"""">https://www.</span><span class="""""""">worldlibertyfinancial.com/</span><span class=""""invisible""""></span></a></p>"""
2,1727190126685,0.0,1.131931e+17,True,0.0,"""<p>RT <span class=""""h-card""""><a href=""""https://truthsocial.com/@realtrumpcoins"""" class=""""u-url mention"""">@<span>realtrumpcoins</span></a></span>The ONLY official coin designed by <span class=""""h-card""""><a href=""""https://truthsocial.com/@realDonaldTrump"""" class=""""u-url mention"""">@<span>realDonaldTrump</span></a></span>"
3,1726957374556,7797.0,1.131779e+17,False,2029.0,"""<p><a href=""""https://links.truthsocial.com/link/113161965355999676"""" rel=""""nofollow noopener noreferrer"""" target=""""_blank""""><span class=""""invisible"""">https://</span><span class=""""ellipsis"""">justthenews.com/politics-polic</span><span class=""""invisible"""">y/all-things-trump/trump-uses-bitcoin-buy-hamburgers-new-york</span></a></p>"""
4,1726954291754,7113.0,1.131777e+17,False,1909.0,"""<p>“Trump Credits Success of His NFTs for Change of Heart on Bitcoin and Crypto” <br/><a href=""""https://links.truthsocial.com/link/113177676465018997"""" rel=""""nofollow noopener noreferrer"""" target=""""_blank""""><span class=""""invisible"""">https://</span><span class=""""ellipsis"""">decrypt.co/249755/trump-credit</span><span class=""""invisible"""">s-nfts-change-of-heart-bitcoin-crypto</span></a></p>"""
...,...,...,...,...,...,...
79,1318015266000,18.0,1.223910e+17,False,58.0,@BarackObama is trying to pass another stimulus after the first was a complete failure. Does he really believe we are that stupid?
80,1316524656000,11.0,1.161389e+17,False,38.0,"""@BarackObama's method of spending the stimulus -- """"Obama fundraiser linked to loan program that aided Solyndra"""" http://t.co/wSiSn5IP"""
81,1315593023000,23.0,1.122314e+17,False,80.0,"@BarackObama---the stimulus will fail, just like the first."
82,1315340614000,105.0,1.111727e+17,False,167.0,@BarackObama trillion $ stimulus was supposed to create jobs -- instead it gave us more debt and higher unemployment.
